In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as tt
import tarfile
# from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder, MNIST
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
class Model(nn.Module):
    def __init__(self, input_sz):
        super().__init__()
        self.input_sz=input_sz
        self.lay1 = nn.Linear(input_sz, 256)
        self.lay4 = nn.Linear(256,64)
        self.lay5 = nn.Linear(64,10)
        
    def forward(self, x):
        x = x.reshape(-1, self.input_sz)
        x = F.relu(self.lay1(x), inplace=True)
        x = F.relu(self.lay4(x), inplace=True)
        y = F.relu(self.lay5(x), inplace=True)
        return y

In [3]:
dataset = MNIST(root='data',download=False, train=True, transform=ToTensor())

In [ ]:
dataset[0]

In [4]:
img, lab = dataset[0]
img.shape

torch.Size([1, 28, 28])

In [13]:
# img = img.resize(1, 784)
model = Model(784)
y = model(img)
y

tensor([[0.0410, 0.0569, 0.1040, 0.0000, 0.0461, 0.0000, 0.1708, 0.0467, 0.0000,
         0.0000]], grad_fn=<ReluBackward1>)

In [ ]:
y = F.softmax(y)
y

In [16]:
loss_fn = F.cross_entropy
# print(loss_fn(prob, labels))

lr = 0.01
opt = torch.optim.SGD(model.parameters(), lr=lr)

def batch_loss(xb, yb, model, loss_fn, opt = None, metrics = None):
    y = model(xb)
    loss = loss_fn(y, yb)
        
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    
    if metrics is not None:
        metric = metrics(y, yb)
    
    return loss.item(), len(xb), metric

def evaluate(model, val, loss_fn, metrics=None):
    losses = [batch_loss(xb, yb, model, loss_fn, metrics=metrics) for xb, yb in val]
    loss, batch_sz, metric = zip(*losses)
    
    total = np.sum(batch_sz)
    
    avg_loss = np.sum(np.multiply(loss, batch_sz)) / total
    
    if metrics is not None:
        avg_met = np.sum(np.multiply(metric, batch_sz)) / total
        
    return avg_loss, total, avg_met

def accuracy(out, labels):
    _, preds = torch.max(out, dim=1)
    return torch.sum(preds == labels).item() / len(labels)

In [17]:
model = Model(28*28)

In [ ]:
train_dl = DataLoader(dataset, 128, shuffle=True)

In [10]:
def fit(epochs, model, train_dl, val_dl, opt, loss_fn, metrics=None):
    for epoch in range(epochs):
        for xb, yb in train_dl:
            
            loss, _, _ = batch_loss(xb,yb,model,loss_fn,opt,metrics)
        
        val_loss,total,val_metr = evaluate(model, val_dl, loss_fn, metrics)
        print(f'{epoch+1}/{epochs}:\nloss: {val_loss}')
        if metrics is not None:
              print(f'{metrics.__name__}: {val_metr}')

In [9]:
val_split = 0.2
train_sz, val_sz = int(len(dataset)*(1-val_split)), int(len(dataset)*(val_split))
# print(train_sz, val_sz)

train_ds, val_ds = random_split(dataset, [train_sz, val_sz])
len(train_ds), len(val_ds)

batch_size = 128

train_dl = DataLoader(train_ds, batch_size, shuffle = True)
val_dl = DataLoader(val_ds, batch_size, shuffle = True)

In [18]:
fit(5, model, train_dl, val_dl, opt, loss_fn, accuracy)

1/5:
loss: 2.3034359245300293
accuracy: 0.11008333333333334
2/5:
loss: 2.3034358806610107
accuracy: 0.11008333333333334
3/5:
loss: 2.303435879389445
accuracy: 0.11008333333333334
4/5:
loss: 2.3034359194437664
accuracy: 0.11008333333333334
5/5:
loss: 2.3034358921051026
accuracy: 0.11008333333333334


In [ ]:
for x,y in train_dl:
    print(x.shape)
    break